In [33]:
import pandas as pd

In [2]:
s3_endpoint = "http://localhost:9000" 

access_key = "accesskey"
secret_key = "secretkey"

bronze_bucket = "bronze"
silver_bucket = "silver"
gold_bucket = "gold"

s3fs_opts = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {"endpoint_url": s3_endpoint},
}




In [25]:
def df_nan_percentage(df):
    """Return a Series showing percentage of NaN values per column."""

    return df.isna().mean().mul(100).round(2).sort_values(ascending=False).map(lambda x: f"{x}%")


# Raw Data Sample

In [ ]:
bios_raw_data_df = pd.read_parquet(f"s3://{bronze_bucket}/raw_data/biodata.parquet", storage_options=s3fs_opts)
editions_raw_data_df = pd.read_parquet(f"s3://{bronze_bucket}/raw_data/editions.parquet", storage_options=s3fs_opts)
results_raw_data_df = pd.read_parquet(f"s3://{bronze_bucket}/raw_data/results.parquet", storage_options=s3fs_opts)

In [17]:
bios_raw_data_df.sample(5)

,Roles,Sex,Full name,Used name,Born,Died,NOC,Athlete_Id,Affiliations,Nick/petnames,Measurements,Title(s),Nationality,Other names,Original name,Name order
107268,Competed in Olympic Games,Male,Keng•Po-Hsuan,Keng•Po-Hsuan,"15 October 1984 in Taipei, Taiwan (TPE)",<NA>,Chinese Taipei,108398,"Fubon Bulls, Taipei (TPE)",<NA>,185 cm / 95 kg,<NA>,<NA>,<NA>,耿•伯軒,Oriental
60099,Competed in Olympic Games,Male,Artyom Sarkisovich•Teryan,Artyom•Teryan,"5 March 1930 in Gəncə, Gəncə (AZE)","April 1970 in Bakı, Bakı (AZE)",Soviet Union,60547,"Dynamo Bakı, Bakı (AZE)",<NA>,<NA>,<NA>,Azerbaijan,<NA>,Артём Саркисович•Терян,<NA>
87458,Competed in Olympic Games,Male,David•Chelule,David•Chelule,7 July 1977,<NA>,Kenya,88158,<NA>,<NA>,179 cm / 63 kg,<NA>,<NA>,<NA>,<NA>,<NA>
13393,Competed in Olympic Games,Male,Antonio•Bailetti,Antonio•Bailetti,"29 September 1937 in Nanto, Vicenza (ITA)",<NA>,Italy,13475,"Gruppo Sportivo Cademartori, Mariano Comense (...",Toni,179 cm / 70 kg,<NA>,<NA>,<NA>,<NA>,<NA>
113548,Competed in Olympic Games,Female,Blake•Russell (Phillips-),Blake•Russell,"24 July 1975 in Winston-Salem, North Carolina ...",<NA>,United States,114852,Reebok,<NA>,165 cm / 48 kg,<NA>,<NA>,<NA>,<NA>,<NA>


In [5]:

results_raw_data_df.sample(5)

,Games,Event,Team,Pos,Medal,As,NOC,Discipline,Athlete_Id,Nationality,Unnamed: 7
227827,2004 Summer Olympics,"Hammer Throw, Women (Olympic)",<NA>,9,<NA>,Candice Scott,TTO,Athletics,105348,<NA>,None
15927,1992 Summer Olympics,"Kayak Fours, 1,000 metres, Men (Olympic)",Italy,6 h2 r2/3,<NA>,Matteo Bruscoli,ITA,Canoe Sprint (Canoeing),10161,<NA>,None
157622,1992 Summer Olympics,"Shot Put, Women (Olympic)",<NA>,1,Gold,Svetlana Krivelyova,EUN,Athletics,75365,<NA>,None
95655,1912 Summer Olympics,"Small-Bore Rifle, Any Position, 50 metres, Men...",<NA>,1,Gold,Fred Hird,USA,Shooting,44688,<NA>,None
194248,2000 Summer Olympics,"Football, Men (Olympic)",Kuwait,12,<NA>,Adel Okla,KUW,Football (Football),91121,<NA>,None


In [6]:
editions_raw_data_df.head()

,#,Year,City,Country,Opened,Closed,Competition,Unnamed: 7,Game_Type,Edition_Name
0,I,1896,Athina,GRE,6 April,15 April,6 – 13 April,<NA>,Olympic Games,Summer
1,II,1900,Paris,FRA,<NA>,<NA>,14 May – 28 October,<NA>,Olympic Games,Summer
2,III,1904,St. Louis,USA,14 May,<NA>,1 July – 26 November,<NA>,Olympic Games,Summer
3,IV,1908,London,GBR,13 July,25 July,27 April – 31 October,<NA>,Olympic Games,Summer
4,V,1912,Stockholm,SWE,6 July,15 July,5 May – 27 July,<NA>,Olympic Games,Summer


# Cleaned Data Stage I

In [7]:
cleaned_biodata_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data/cleaned_biodata.parquet", storage_options=s3fs_opts)
cleaned_affiliations_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data/dim_affiliation.parquet", storage_options=s3fs_opts)
bridge_athlete_affiliation_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data/bridge_athlete_affiliation.parquet", storage_options=s3fs_opts)
cleaned_editions_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data/cleaned_editions.parquet", storage_options=s3fs_opts)
cleaned_results_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data/cleaned_results.parquet", storage_options=s3fs_opts)

In [18]:
cleaned_biodata_df.sample(5)

,Roles,Sex,NOC,Athlete_Id,Name,Height (cm),Weight (kg),Born_Date,Died_Date,Is_Alive,Born_City,Born_Region,Born_Country
70086,Competed in Olympic Games,Male,ghana,70627,James Addy,183.0,75.0,1939-12-09,2009-04-26,False,Accra,Greater Accra,GHA
139972,Competed in Olympic Games,Male,germany,143419,Felix Uduokhai,192.0,NaN,1997-09-09,NaT,True,Annaberg-Buchholz,Sachsen,GER
80022,Competed in Olympic Games,Female,slovenia,80689,Nataša Bokal,172.0,68.0,1967-05-09,NaT,True,<NA>,<NA>,<NA>
36018,Competed in Olympic Games,Male,canada,36304,Bill Wood,NaN,NaN,1899-01-06,1969-10-02,False,Winnipeg,Manitoba,CAN
104542,Competed in Olympic Games,Male,ukraine,105570,Mykola Savolainen,190.0,75.0,1980-03-25,NaT,True,Kyiv,Kyiv,UKR


In [23]:
cleaned_affiliations_df.sample(5)

,Affiliation_Id,Affiliation_Club,Affiliation_City,Affiliation_Country
28275,28275,Riccardi Milano,None,None
27204,27204,Houston Dynamo,Houston,USA
26,26,NFD Grove,None,None
26842,26842,Dynamo Verkhnyaya Pyshma,Verkhnyaya Pyshma,RUS
20042,20042,Brunnsvikens SK,Stockholm,SWE


In [20]:
bridge_athlete_affiliation_df.sample(5)

,Athlete_Id,Affiliation_Id
117378,111054,677
87571,83597,8322
13468,12862,3426
33131,31820,0
8380,7921,0


In [21]:
cleaned_editions_df.sample(5)

,Year,City,Country,Opened,Closed,Comments,Game_Type,Edition_Name,Competition_Start,Competition_End,Game_Id
45,1988,Seoul,KOR,1988-09-17,1988-10-02,<NA>,Olympic Games,Summer,1988-09-17,1988-10-02,46
18,1936,Berlin,GER,1936-08-01,1936-08-16,<NA>,Olympic Games,Summer,1936-08-01,1936-08-16,19
46,1988,Calgary,CAN,1988-02-13,1988-02-28,<NA>,Olympic Games,Winter,1988-02-13,1988-02-28,47
52,2000,Sydney,AUS,2000-09-15,2000-10-01,<NA>,Olympic Games,Summer,2000-09-13,2000-10-01,53
33,1964,Tokyo,JPN,1964-10-10,1964-10-24,<NA>,Olympic Games,Summer,1964-10-11,1964-10-24,34


In [24]:
cleaned_results_df.sample(5)

,Event,Team,Medal,As,NOC,Discipline,Athlete_Id,Game Year,Game Type,Position,Tied
75333,"Half-Heavyweight, Women (Olympic)",<NA>,<NA>,Edinanci da Silva,BRA,Judo,33149,2000,Summer Olympics,7,True
47946,"Football, Men (Olympic)",Unified Team of Germany,Bronze,Werner Unger,GER,Football (Football),25356,1964,Summer Olympics,3,False
69771,"Javelin Throw, Freestyle, Men (Intercalated)",<NA>,<NA>,Erik Granfelt,SWE,Tug-Of-War,31078,1906,Intercalated Games,<NA>,False
79965,"Eights, Men (Olympic)",Toronto Argonauts,Silver,Alan Bailey,CAN,Rowing,36074,1904,Summer Olympics,2,False
276965,"Individual, Women (Olympic)",<NA>,<NA>,Sarah Nikitin,BRA,Archery,129215,2016,Summer Olympics,33,True


# Cleaned Data Stage II

In [8]:
imputed_bios_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data_II/cleaned_biodata.parquet", storage_options=s3fs_opts)
imputed_editions_df = pd.read_parquet(f"s3://{silver_bucket}/clean_data_II/cleaned_editions.parquet", storage_options=s3fs_opts)

In [26]:
imputed_bios_df.sample(5)

,Roles,Sex,NOC,Athlete_Id,Name,Height (cm),Weight (kg),Born_Date,Died_Date,Is_Alive,Born_City,Born_Region,Born_Country,Height_Imputed,Weight_Imputed,Born_Country_From_NOC
97833,Competed in Olympic Games,Male,romania,98627,Nicolae Veștea,181.0,72.0,1950-07-11,NaT,True,<NA>,<NA>,ROU,False,False,True
59699,Competed in Olympic Games,Male,sweden,60140,Frans Westergren,172.0,72.0,1914-04-20,1993-03-02,False,Malmö,Skåne,SWE,True,True,False
2274,"Competed in Olympic Games,Other",Male,denmark,2391,Thomas Stuer-Lauridsen,191.0,78.0,1971-04-29,NaT,True,Hørsholm,Hovedstaden,DEN,False,False,False
127381,Competed in Olympic Games,Female,colombia,129781,Khaterinne Medina,162.0,60.0,1992-10-31,NaT,True,Riohacha,La Guajira,COL,False,False,False
51983,Competed in Olympic Games,Female,germany,52362,Constance Radfan,183.0,71.0,1969-10-24,NaT,True,Sassnitz,Mecklenburg-Vorpommern,GER,False,False,False


In [27]:
imputed_editions_df.sample(5)

,Year,City,Country,Opened,Closed,Comments,Game_Type,Edition_Name,Competition_Start,Competition_End,Game_Id,Opened_Imputed,Closed_Imputed,Competition_Start_Imputed,Competition_End_Imputed
56,2008,Beijing,CHN,2008-08-08,2008-08-24,<NA>,Olympic Games,Summer,2008-08-06,2008-08-24,57,False,False,False,False
20,1940,Helsinki,FIN,None,None,Not held due to war,Olympic Games,Summer,None,None,21,False,False,False,False
66,2018,Buenos Aires,ARG,2018-10-06,2018-10-18,<NA>,Youth Olympic Games,Summer,2018-10-06,2018-10-18,67,False,False,False,False
59,2012,London,GBR,2012-07-27,2012-08-12,<NA>,Olympic Games,Summer,2012-07-25,2012-08-12,60,False,False,False,False
7,1906,Athina,GRE,1906-04-22,1906-05-02,<NA>,Intercalated Games,,1906-04-22,1906-05-02,8,False,False,False,False


# Missing Values Percentage Comparison

In [44]:
bios1 = pd.Series(df_nan_percentage(cleaned_biodata_df), name="Before Imputing")
bios2 = pd.Series(df_nan_percentage(imputed_bios_df), name="After Imputing")

games1 = pd.Series(df_nan_percentage(cleaned_editions_df), name="Before Imputing")
games2 = pd.Series(df_nan_percentage(imputed_editions_df), name="After Imputing")

**Bios Data:**

In [43]:
pd.concat([bios1, bios2], axis=1)

,Before Imputing,After Imputing
Died_Date,76.67%,76.67%
Weight (kg),29.85%,0.0%
Height (cm),26.69%,0.0%
Born_City,22.67%,22.67%
Born_Country,22.67%,4.56%
Born_Region,22.67%,22.67%
Born_Date,1.24%,1.24%
Name,0.0%,0.0%
Sex,0.0%,0.0%
Roles,0.0%,0.0%


**Games Data:**

In [65]:
pd.concat([games1, games2], axis=1)


,Before Imputing,After Imputing
Comments,93.42%,93.42%
Closed,21.05%,15.79%
Opened,19.74%,14.47%
Competition_End,15.79%,15.79%
Competition_Start,14.47%,14.47%
Year,0.0%,0.0%
City,0.0%,0.0%
Country,0.0%,0.0%
Game_Type,0.0%,0.0%
Edition_Name,0.0%,0.0%


# Data Quality Failure Cases

In [46]:
bios_failure_cases_df = pd.read_parquet(f"s3://{silver_bucket}/failure_cases/bios_failure_cases.parquet", storage_options=s3fs_opts)
affiliations_failure_cases_df = pd.read_parquet(f"s3://{silver_bucket}/failure_cases/affiliations_failure_cases.parquet", storage_options=s3fs_opts)
editions_failure_cases_df = pd.read_parquet(f"s3://{silver_bucket}/failure_cases/editions_failure_cases.parquet", storage_options=s3fs_opts)
results_failure_cases_df = pd.read_parquet(f"s3://{silver_bucket}/failure_cases/results_failure_cases.parquet", storage_options=s3fs_opts)

In [56]:
bios_failure_cases_df.sample(5)

,Athlete_Id,Born_Country_From_NOC,Height (cm),Height_Imputed,Is_Alive,NOC,Name,Roles,Sex,Weight (kg),Weight_Imputed,failed_check,Born_Date,Died_Date,Born_City,Born_Region,Born_Country
16,28313,False,158.0,False,True,spain,Tania Lamarca,Competed in Olympic Games,Female,36.0,False,height_weight_ratio_invalid,1980-04-30,NaT,Vitoria,Álava,ESP
45,91334,False,168.0,False,True,germany,Friederike Arlt,Competed in Olympic Games,Female,42.0,False,height_weight_ratio_invalid,1984-09-07,NaT,Freiberg,Sachsen,GER
78,131458,False,179.0,False,True,puerto rico,Yarimar Mercado,Competed in Olympic Games,Female,48.0,False,height_weight_ratio_invalid,1995-03-12,NaT,Yauco,Puerto Rico,PUR
67,116905,False,168.0,False,True,ukraine,Krystyna Cherepienina,Competed in Olympic Games,Female,42.0,False,height_weight_ratio_invalid,1991-11-21,NaT,Luhansk,Luhansk,UKR
2,80315,True,168.0,True,True,france,Georges Dubois,Competed in Olympic Games,Male,63.0,True,duplicate_name_birth_date,NaT,NaT,<NA>,<NA>,FRA


In [53]:
affiliations_failure_cases_df.sample(5)

,Affiliation_Club,Affiliation_Id,failed_check,Affiliation_City,Affiliation_Country
1128,Wrestling Club,22498,duplicate_affiliation_with_different_ids,Bagnolet,None
2002,St Peters Western,32745,duplicate_affiliation_with_different_ids,None,"Brisbane, QLD, AUS"
2374,AS Volley Lube,36048,duplicate_affiliation_with_different_ids,None,ITA
569,APEF Milano,13431,duplicate_affiliation_with_different_ids,None,None
1522,Corpo Forestale Stato,28824,duplicate_affiliation_with_different_ids,None,None


In [57]:
editions_failure_cases_df.sample(5)

,Edition_Name,failed_check,Year,City,Country,Opened,Closed,Comments,Game_Type,Competition_Start,Competition_End,Game_Id,Opened_Imputed,Closed_Imputed,Competition_Start_Imputed,Competition_End_Imputed
10,,competition_start_after_end,1859,Athina,GRE,None,None,<NA>,Forerunners to the Olympic Games,None,None,1,False,False,False,False
4,,"isin(['Summer', 'Winter', 'Equestrian'])",1906,Athina,GRE,1906-04-22,1906-05-02,<NA>,Intercalated Games,1906-04-22,1906-05-02,8,False,False,False,False
9,,opened_after_closed,1906,Athina,GRE,1906-04-22,1906-05-02,<NA>,Intercalated Games,1906-04-22,1906-05-02,8,False,False,False,False
14,,competition_start_after_end,1906,Athina,GRE,1906-04-22,1906-05-02,<NA>,Intercalated Games,1906-04-22,1906-05-02,8,False,False,False,False
1,,"isin(['Summer', 'Winter', 'Equestrian'])",1870,Athina,GRE,None,None,<NA>,Forerunners to the Olympic Games,None,None,2,False,False,False,False


In [50]:
results_failure_cases_df.sample(5)

,As,Athlete_Id,Discipline,Event,NOC,Position,Tied,failed_check,Team,Medal,Game Year,Game Type
6077,Georgios Parris,70828,Athletics,"400 metres Hurdles, Men (Olympic)",GRE,1,False,position_medal_mismatch,<NA>,<NA>,1976,Summer Olympics
6186,Zoltán Speidl,71284,Athletics,"400 metres, Men (Olympic)",HUN,3,False,position_medal_mismatch,<NA>,<NA>,1900,Summer Olympics
225,Peter Charles,11747,Equestrian Jumping (Equestrian),"Individual, Open (Olympic)",IRL,2,False,position_medal_mismatch,Kruger,<NA>,1992,Summer Olympics
1527,Pedro Cabrera,21640,Fencing,"Épée, Team, Men (Olympic)",ESP,3,False,position_medal_mismatch,Spain,<NA>,1960,Summer Olympics
5230,Marita Payne Wiggins,65899,Athletics,"4 × 400 metres Relay, Women (Olympic)",CAN,2,False,position_medal_mismatch,Canada,<NA>,1988,Summer Olympics


# Final Cleaned Warehouse Modeld Data 

In [11]:
dim_athletes_df=pd.read_parquet(f"s3://{gold_bucket}/clean_data_final/dim_athletes.parquet", storage_options=s3fs_opts)
dim_affiliations_df=pd.read_parquet(f"s3://{gold_bucket}/clean_data_final/dim_affiliations.parquet", storage_options=s3fs_opts)
bridge_athletes_affiliations_df=pd.read_parquet(f"s3://{gold_bucket}/clean_data_final/bridge_athletes_affiliations.parquet", storage_options=s3fs_opts)
dim_games_df=pd.read_parquet(f"s3://{gold_bucket}/clean_data_final/dim_games.parquet", storage_options=s3fs_opts)
fct_results_df=pd.read_parquet(f"s3://{gold_bucket}/clean_data_final/fct_results.parquet", storage_options=s3fs_opts)


In [64]:
dim_athletes_df.sample(5)

,athlete_id,athlete_name,athlete_roles,athlete_sex,athlete_NOC,athlete_height_cm,athlete_weight_kg,athlete_born_date,athlete_died_date,athlete_is_alive,athlete_born_city,athlete_born_region,athlete_born_country,athlete_is_height_imputed,athlete_is_weight_imputed,athlete_is_born_country_from_NOC
10210,10265,Jeong Mi,Competed in Olympic Games,Female,republic of korea,163.0,58.0,1970-07-27,NaT,True,<NA>,<NA>,None,False,False,True
134968,138041,Erwan Käser,Competed in Olympic Games,Male,switzerland,177.0,68.0,1992-06-08,NaT,True,<NA>,<NA>,CHE,False,False,True
141203,144695,Dai Yoshioka,Competed in Olympic Games,Male,japan,175.0,76.0,1985-12-14,NaT,True,<NA>,<NA>,JPN,True,True,True
131483,134229,Oliver Szymanski,Competed in Olympic Games,Male,germany,183.0,73.0,1990-07-27,NaT,True,Berlin,Berlin,GER,False,False,False
49133,49502,Martin van der Spoel,Competed in Olympic Games,Male,netherlands,184.0,77.0,1971-02-19,NaT,True,Ermelo,Gelderland,NED,True,True,False


In [59]:
dim_affiliations_df.sample(5)

,affiliation_id,dim_affiliation_club,dim_affiliation_city,dim_affiliation_country
24662,24662,Tri Club Zofingen,None,None
8563,8563,Judo Club Kufstein,None,None
13703,13703,Baník Prievidza,None,None
27940,27940,Dream Seeker Racing Team,None,None
4154,4154,Leichtathletik Gemeinschaft Stuttgart,None,None


In [60]:
bridge_athletes_affiliations_df.sample(5)

,athlete_id,affiliation_id
13307,12705,3595
34755,33344,0
20148,19285,5210
151420,145311,28042
109342,103678,20933


In [61]:
dim_games_df.sample(5)

,game_id,dim_game_type,dim_edition_name,dim_game_year,dim_city,dim_country,dim_opened,dim_closed,dim_competition_start,dim_competition_end,dim_comments,dim_opened_imputed,dim_closed_imputed,dim_competition_start_imputed,dim_competition_end_imputed
14,15,Olympic Games,Summer,1928,Amsterdam,NED,1928-07-28,1928-08-12,1928-05-17,1928-08-12,<NA>,False,False,False,False
9,10,Olympic Games,Summer,1912,Stockholm,SWE,1912-07-06,1912-07-15,1912-05-05,1912-07-27,<NA>,False,False,False,False
26,27,Olympic Games,Summer,1952,Helsinki,FIN,1952-07-19,1952-08-03,1952-07-14,1952-08-03,<NA>,False,False,False,False
23,24,Olympic Games,Winter,1944,Cortina d'Ampezzo,ITA,None,None,None,None,Not held due to war,False,False,False,False
8,9,Olympic Games,Summer,1908,London,GBR,1908-07-13,1908-07-25,1908-04-27,1908-10-31,<NA>,False,False,False,False


In [62]:
fct_results_df.sample(5)

,athlete_id,dim_noc,dim_discipline,dim_game_type,dim_game_year,dim_event_name,dim_team_name,dim_as,m_tied_flag,m_position,m_medal
294599,138787,ITA,Biathlon,Winter Youth Olympics,2020,"12.5 kilometres, Boys (YOG)",<NA>,Daniel Oberegger,False,62,<NA>
68650,30955,ROU,Artistic Gymnastics (Gymnastics),Summer Olympics,1952,"Horizontal Bar, Men (Olympic)",<NA>,Friedrich Orendi,True,51,<NA>
210304,98150,FRA,Bobsleigh (Bobsleigh),Winter Olympics,1994,"Two, Men (Olympic)",Gabriel Fourmigue,Philippe Tanchon,False,23,<NA>
138068,66125,CAN,Athletics,Summer Olympics,1920,"5,000 metres, Men (Olympic)",<NA>,Eddie Lawrence,False,<NA>,<NA>
259392,120352,RUS,Archery,Summer Olympics,2012,"Team, Women (Olympic)",Russian Federation,Kseniya Perova,False,4,<NA>
